In [69]:
import pandas as pd
import numpy as np
import re
data=pd.read_csv('artifacts/Cleaned_Indian_Food_Dataset.csv',usecols=['TranslatedRecipeName','Cleaned-Ingredients','TranslatedInstructions','Ingredient-count'])

In [70]:
def clean_ingredients(ingredient_string):
    # Remove anything inside parentheses
    cleaned_string = re.sub(r'\s*\(.*?\)\s*', '', ingredient_string)
    # Convert the cleaned string to a list
    return [item.strip() for item in cleaned_string.split(',')]

# Apply the function to the 'ingredients' column and create a new column 'Ingredients cleaned'
data['Processed_Ingredients'] = data['Cleaned-Ingredients'].apply(clean_ingredients)

In [72]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms.ollama import Ollama
llm = Ollama(model='llama3.2')

prompt=ChatPromptTemplate([
        ('system', """
        You are a master Indian chef specializing in traditional Indian cuisine. The user will give you a list of ingredients, and you are required to create a genuine Indian dish using only those ingredients.
        Use the following structure for your response:
        
        Ingredients Used:
            - Ingredient 1
            - Ingredient 2
            - (List all ingredients provided by the user)
        
        Instructions:
            - Step 1: (Describe each step of the cooking process in detail)
            - Step 2: (Continue outlining the steps until the dish is complete)
     
        Below are sample instructions for a dish provided. Form your sentences in a similar manner.
        - Step 1: To prepare gourd raita, prepare all the ingredients first.
        - Step 2: Add grated gourd, cucumber, curd, green chillies, salt, cumin powder and coriander in a large bowl.
        - Step 3: Mix well and your raita is ready.
        - Step 4: Serve gourd raita with Garlic Dal, gourd elder greens and phulka for dinner.
        
        Keep your response focused on 'Ingredients Used' and 'Instructions'. Avoid adding additional explanations or extra information.
    """),
        ("user","Ingredients: {Ingredients}")

    ])

chain = prompt | llm | StrOutputParser()

response = chain.invoke({'Ingredients': data['Processed_Ingredients'][0]})

In [73]:
response

'Ingredients Used:\n            - Salt\n            - Amchur\n            - Karela\n            - Red chilli powder\n            - Gram flour\n            - Onion\n            - Cumin seeds\n            - Coriander powder\n            - Turmeric powder\n            - Sunflower oil\n\nInstructions:\n            - Step 1: Soak the gram flour in water for about 30 minutes to make it smooth and easy to grind.\n            - Step 2: Heat sunflower oil in a pan over medium heat, add cumin seeds and let them sizzle for a few seconds.\n            - Step 3: Add chopped onion and sauté until they are translucent and lightly browned.\n            - Step 4: Grind the soaked gram flour into a fine paste with turmeric powder, red chilli powder, coriander powder, and salt.\n            - Step 5: Add the ground spice mixture to the pan with onion and mix well.\n            - Step 6: Slice the karela into thin rounds, remove excess moisture by salting and draining.\n            - Step 7: Dip the slice

In [74]:
lines = response.split("\n")

gen_ingredients = []
gen_instructions = []

current_section = None

for line in lines:
    stripped_line = line.strip()  
    
    if stripped_line == 'Ingredients Used:':
        current_section = 'ingredients'
        continue  
    elif stripped_line == 'Instructions:':
        current_section = 'instructions'
        continue 
    elif stripped_line.startswith('- '):
        # Extract the item after '- '
        item = stripped_line[2:].strip()
        if current_section == 'ingredients':
            gen_ingredients.append(item)
        elif current_section == 'instructions':
            gen_instructions.append(item)
    else:
        continue

print("Ingredients:")
for idx, ingredient in enumerate(gen_ingredients):
    print(f"{idx}. {ingredient}")

print("\nInstructions:")
for idx, instruction in enumerate(gen_instructions):
    print(f"{idx}. {instruction}")


Ingredients:
0. Salt
1. Amchur
2. Karela
3. Red chilli powder
4. Gram flour
5. Onion
6. Cumin seeds
7. Coriander powder
8. Turmeric powder
9. Sunflower oil

Instructions:
0. Step 1: Soak the gram flour in water for about 30 minutes to make it smooth and easy to grind.
1. Step 2: Heat sunflower oil in a pan over medium heat, add cumin seeds and let them sizzle for a few seconds.
2. Step 3: Add chopped onion and sauté until they are translucent and lightly browned.
3. Step 4: Grind the soaked gram flour into a fine paste with turmeric powder, red chilli powder, coriander powder, and salt.
4. Step 5: Add the ground spice mixture to the pan with onion and mix well.
5. Step 6: Slice the karela into thin rounds, remove excess moisture by salting and draining.
6. Step 7: Dip the sliced karela in the gram flour paste, coating them evenly, then add the amchur powder for extra flavor.
7. Step 8: Heat some oil in a pan over medium heat, add the coated karela slices and fry until they are golden b

In [75]:
data['Processed_Ingredients'][0]

['salt',
 'amchur',
 'karela',
 'red chilli powder',
 'gram flour',
 'onion',
 'cumin seeds',
 'coriander powder',
 'turmeric powder',
 'sunflower oil']

In [76]:
actual_ingredients=data['Processed_Ingredients'][0]
gen_ingredients_lower = [ingredient.lower() for ingredient in gen_ingredients]

total_input_ingredients = len(actual_ingredients)
total_ingredients_used = len(gen_ingredients_lower)

relevant_ingredients_used = len(set(actual_ingredients).intersection(set(gen_ingredients_lower)))

precision = relevant_ingredients_used / total_ingredients_used
recall = relevant_ingredients_used / total_input_ingredients

print(precision)
print(recall)

1.0
1.0


In [77]:
data['TranslatedInstructions'][0].split('/n')

['To begin making the Masala Karela Recipe,de-seed the karela and slice.\nDo not remove the skin as the skin has all the nutrients.\nAdd the karela to the pressure cooker with 3 tablespoon of water, salt and turmeric powder and pressure cook for three whistles.\nRelease the pressure immediately and open the lids.\nKeep aside.Heat oil in a heavy bottomed pan or a kadhai.\nAdd cumin seeds and let it sizzle.Once the cumin seeds have sizzled, add onions and saute them till it turns golden brown in color.Add the karela, red chilli powder, amchur powder, coriander powder and besan.\nStir to combine the masalas into the karela.Drizzle a little extra oil on the top and mix again.\nCover the pan and simmer Masala Karela stirring occasionally until everything comes together well.\nTurn off the heat.Transfer Masala Karela into a serving bowl and serve.Serve Masala Karela along with Panchmel Dal and Phulka for a weekday meal with your family.\n']

In [78]:
gen_instructions

['Step 1: Soak the gram flour in water for about 30 minutes to make it smooth and easy to grind.',
 'Step 2: Heat sunflower oil in a pan over medium heat, add cumin seeds and let them sizzle for a few seconds.',
 'Step 3: Add chopped onion and sauté until they are translucent and lightly browned.',
 'Step 4: Grind the soaked gram flour into a fine paste with turmeric powder, red chilli powder, coriander powder, and salt.',
 'Step 5: Add the ground spice mixture to the pan with onion and mix well.',
 'Step 6: Slice the karela into thin rounds, remove excess moisture by salting and draining.',
 'Step 7: Dip the sliced karela in the gram flour paste, coating them evenly, then add the amchur powder for extra flavor.',
 'Step 8: Heat some oil in a pan over medium heat, add the coated karela slices and fry until they are golden brown and crispy.']

In [79]:
data['TranslatedInstructions'][0]

'To begin making the Masala Karela Recipe,de-seed the karela and slice.\nDo not remove the skin as the skin has all the nutrients.\nAdd the karela to the pressure cooker with 3 tablespoon of water, salt and turmeric powder and pressure cook for three whistles.\nRelease the pressure immediately and open the lids.\nKeep aside.Heat oil in a heavy bottomed pan or a kadhai.\nAdd cumin seeds and let it sizzle.Once the cumin seeds have sizzled, add onions and saute them till it turns golden brown in color.Add the karela, red chilli powder, amchur powder, coriander powder and besan.\nStir to combine the masalas into the karela.Drizzle a little extra oil on the top and mix again.\nCover the pan and simmer Masala Karela stirring occasionally until everything comes together well.\nTurn off the heat.Transfer Masala Karela into a serving bowl and serve.Serve Masala Karela along with Panchmel Dal and Phulka for a weekday meal with your family.\n'

In [111]:
import re
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.tokenize import word_tokenize
import nltk

nltk.download('punkt')

# reference_text = data['TranslatedInstructions'][0]
# reference_sentences = re.split(r'[.\n]', reference_text)

# reference_tokens = [word_tokenize(sent.lower()) for sent in reference_sentences if sent.strip()]
# reference_tokens_flat = [token for sent in reference_tokens for token in sent]

# candidate_sentences = gen_instructions
# candidate_tokens = [word_tokenize(sent.lower()) for sent in candidate_sentences]

# candidate_tokens_flat = [token for sent in candidate_tokens for token in sent]


reference_tokens = data['TranslatedInstructions'][0].split('/n')
candidate_text='\n '.join(gen_instructions)
candidate_tokens = candidate_text
weights = (0.5, 0.5)
bleu_score = sentence_bleu(reference_tokens, candidate_tokens,weights=weights)

# weights = (1.0, 0)

# smooth = SmoothingFunction().method4

# bleu_score = sentence_bleu(
#     [reference_tokens_flat],  
#     candidate_tokens_flat,    
#     weights=weights,
#     smoothing_function=smooth
# )

print(f"BLEU Score: {bleu_score:.4f}")


BLEU Score: 0.7126


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [17]:
ingredients = re.findall(r'\* (.+)', response)

In [18]:
ingredients

['Salt',
 'Amchur',
 'Karela',
 'Red chilli powder',
 'Gram flour',
 'Onion',
 'Cumin seeds',
 'Coriander powder',
 'Turmeric powder',
 'Heat sunflower oil in a pan over medium heat.',
 'Add cumin seeds and let them sizzle for a few seconds.',
 'Add finely chopped onion and sauté until it is translucent.',
 'Add karela (bitter gourd) and sauté for 2-3 minutes, or until it starts to soften.',
 'Add salt, red chilli powder, coriander powder, and turmeric powder, and mix well.',
 'Add gram flour (besan) and mix well.',
 'Add amchur (dried mango powder) and mix well.',
 'Cover the pan and cook for 5-7 minutes, or until the karela is cooked through and the mixture is dry.',
 'Serve hot with rice or roti.']

In [5]:
data['Processed_Ingredients'][0]

['salt',
 'amchur',
 'karela',
 'red chilli powder',
 'gram flour',
 'onion',
 'cumin seeds',
 'coriander powder',
 'turmeric powder',
 'sunflower oil']

In [21]:
data['TranslatedInstructions'][0]

'To begin making the Masala Karela Recipe,de-seed the karela and slice.\nDo not remove the skin as the skin has all the nutrients.\nAdd the karela to the pressure cooker with 3 tablespoon of water, salt and turmeric powder and pressure cook for three whistles.\nRelease the pressure immediately and open the lids.\nKeep aside.Heat oil in a heavy bottomed pan or a kadhai.\nAdd cumin seeds and let it sizzle.Once the cumin seeds have sizzled, add onions and saute them till it turns golden brown in color.Add the karela, red chilli powder, amchur powder, coriander powder and besan.\nStir to combine the masalas into the karela.Drizzle a little extra oil on the top and mix again.\nCover the pan and simmer Masala Karela stirring occasionally until everything comes together well.\nTurn off the heat.Transfer Masala Karela into a serving bowl and serve.Serve Masala Karela along with Panchmel Dal and Phulka for a weekday meal with your family.\n'

In [3]:
data['Processed_Ingredients'][0]

['salt',
 'amchur',
 'karela',
 'red chilli powder',
 'gram flour',
 'onion',
 'cumin seeds',
 'coriander powder',
 'turmeric powder',
 'sunflower oil']

In [4]:
from langchain import LLMChain, PromptTemplate
from transformers import pipeline
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

access_token = ""

# Specify the model
model_name = "openai-community/gpt2"

# Try loading the fast tokenizer explicitly
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, use_auth_token=access_token)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, use_auth_token=access_token)


# Define a custom prompt template for generating recipes
recipe_template = """
I have the following ingredients: {ingredients}.
I want to prepare a dish from {cuisine} cuisine.
Please generate a recipe using these ingredients with instructions.

Recipe:
"""

prompt = PromptTemplate(
    input_variables=["ingredients", "cuisine"],
    template=recipe_template
)

# LangChain LLM wrapper for Hugging Face model
class Llama2LLMChain(LLMChain):
    def __init__(self, model, tokenizer):
        super().__init__(llm=pipeline("text-generation", model=model, tokenizer=tokenizer))

    def _call(self, inputs):
        return super()._call(inputs)

# Initialize the LLaMA model as part of LangChain
llama_chain = Llama2LLMChain(model=model, tokenizer=tokenizer)


def generate_recipe(ingredients: list, cuisine: str) -> str:
    # Convert the ingredients list into a comma-separated string
    ingredients_str = ', '.join(ingredients)
    
    # Prepare input data for the LangChain
    input_data = {
        "ingredients": ingredients_str,
        "cuisine": cuisine
    }
    
    # Generate the recipe using the LLaMA model
    response = llama_chain(input_data)
    
    # Extract and return the generated text
    return response['text']

ingredients = data['Processed_Ingredients'][0]
cuisine = 'Indian'

recipe = generate_recipe(ingredients, cuisine)
print(recipe)



c:\Users\arnab\.conda\envs\data_science\Lib\site-packages\transformers\models\auto\tokenization_auto.py:786: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
c:\Users\arnab\.conda\envs\data_science\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
c:\Users\arnab\.conda\envs\data_science\Lib\site-packages\transformers\models\auto\auto_factory.py:469: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


ValidationError: 3 validation errors for Llama2LLMChain
prompt
  Field required [type=missing, input_value={'llm': <transformers.pip... at 0x000001E5AA1EA1E0>}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.7/v/missing
llm.is-instance[Runnable]
  Input should be an instance of Runnable [type=is_instance_of, input_value=<transformers.pipelines.t...t at 0x000001E5AA1EA1E0>, input_type=TextGenerationPipeline]
    For further information visit https://errors.pydantic.dev/2.7/v/is_instance_of
llm.is-instance[Runnable]
  Input should be an instance of Runnable [type=is_instance_of, input_value=<transformers.pipelines.t...t at 0x000001E5AA1EA1E0>, input_type=TextGenerationPipeline]
    For further information visit https://errors.pydantic.dev/2.7/v/is_instance_of